# Detect explicit, suggestive and violent content using Amazon Rekognition

This notebook provides a walkthrough of [content moderation APIs](https://docs.aws.amazon.com/rekognition/latest/dg/moderation.html) in Amazon Rekognition. You can quickly identify inappropriate content in your video and image libraries.

## References
- https://docs.aws.amazon.com/rekognition/latest/dg/API_DetectModerationLabels.html
- https://docs.aws.amazon.com/rekognition/latest/dg/API_StartContentModeration.html
- https://docs.aws.amazon.com/rekognition/latest/dg/API_GetContentModeration.html

## Check dependencies setup

In [2]:
%store -r setup_dependencies_passed

In [4]:
%store -r setup_dependencies_passed

try:
    setup_dependencies_passed
except NameError:
    print("++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] YOU HAVE TO RUN THE PREVIOUS NOTEBOOK ")
    print("You did not install the required libraries.   ")
    print("++++++++++++++++++++++++++++++++++++++++++++++")

if not setup_dependencies_passed:
    print("++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] YOU HAVE TO RUN THE PREVIOUS NOTEBOOK ")
    print("You did not install the required libraries.   ")
    print("++++++++++++++++++++++++++++++++++++++++++++++")
else:
    print("[OK] Dependencies correctly set up")

[OK] Dependencies correctly set up


## Setup notebook

In [23]:
import boto3
import sagemaker
from IPython.display import Image as IImage, display
from IPython.display import HTML, display
from PIL import Image, ImageDraw, ImageFont
import time
import os

In [24]:
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()
bucket = sagemaker_session.default_bucket()
region = boto3.Session().region_name
rekognition = boto3.client("rekognition")
s3 = boto3.client("s3")

# Content Moderation in Video

Content Moderation in video is an async operation. 
https://docs.aws.amazon.com/rekognition/latest/dg/API_StartContentModeration.html
 - We first start content moderation job which returns a Job Id.
 - We can then call `get_content_moderation` to get the job status and after job is complete, we can get moderation results.
 - In production use cases, you would usually use StepFucntion or SNS topic to get notified when job is complete.

## Get video and upload to S3

In [33]:
media_folder = "../media"

In [44]:
video_name = "weapon.mp4"

str_detail = "Moderation labels in video<br>=======================================<br>"
str_overall = "Moderation labels in the overall video:<br>=======================================<br>"

In [35]:
s3.upload_file(
    f"{media_folder}/{video_name}",
    bucket,
    video_name
)

In [36]:
s3VideoUrl = s3.generate_presigned_url("get_object", Params={"Bucket": bucket, "Key": videoName})

videoTag = "<video controls='controls' autoplay width='640' height='360' name='Video' src='{0}'></video>".format(
    s3VideoUrl
)

videoui = "<table><tr><td style='vertical-align: top'>{}</td></tr></table>".format(videoTag)

display(HTML(videoui))

# Call Rekognition to Start a Job for Content Moderation

### Additional (Optional) Request Attributes

ClientRequestToken:
https://docs.aws.amazon.com/rekognition/latest/dg/API_StartContentModeration.html#rekognition-StartContentModeration-request-ClientRequestToken

JobTag:
https://docs.aws.amazon.com/rekognition/latest/dg/API_StartContentModeration.html#rekognition-StartContentModeration-request-JobTag

MinConfidence:
https://docs.aws.amazon.com/rekognition/latest/dg/API_StartContentModeration.html#rekognition-StartContentModeration-request-MinConfidence

NotificationChannel:
https://docs.aws.amazon.com/rekognition/latest/dg/API_StartContentModeration.html#rekognition-StartContentModeration-request-NotificationChannel


In [48]:
# Start content moderation job
start_moderation_label_detection = rekognition.start_content_moderation(
    Video={
        "S3Object": {
            "Bucket": bucket,
            "Name": video_name,
        }
    },
    MinConfidence=50.0
)

moderation_job_id = start_moderation_label_detection["JobId"]
display("Job Id: {0}".format(moderation_job_id))

'Job Id: 88f3510c6ee1c55c0b5594bcbbf67b0288c137c7d973f627d8383b794dbef551'

# Wait for content moderation job to complete
In production use cases, you would usually use StepFunction or SNS topic to get notified when job is complete.


In [49]:
%%time

get_content_moderation = rekognition.get_content_moderation(JobId=moderation_job_id, SortBy="TIMESTAMP")

while get_content_moderation["JobStatus"] == "IN_PROGRESS":
    time.sleep(5)
    print(".", end="")

    get_content_moderation = rekognition.get_content_moderation(JobId=moderation_job_id, SortBy="TIMESTAMP")

display(get_content_moderation["JobStatus"])

...

'SUCCEEDED'

CPU times: user 13.2 ms, sys: 4.32 ms, total: 17.5 ms
Wall time: 15.2 s


# Review JSON response returned by Rekognition Content Moderation API

In the JSON response below, you will see list of detected content.

For each detected object, you will see `Timestamp`.


In [50]:
display(get_content_moderation)

{'JobStatus': 'SUCCEEDED',
 'VideoMetadata': {'Codec': 'h264',
  'DurationMillis': 6033,
  'Format': 'QuickTime / MOV',
  'FrameRate': 30.0,
  'FrameHeight': 1080,
  'FrameWidth': 1920,
  'ColorRange': 'LIMITED'},
 'ModerationLabels': [{'Timestamp': 1000,
   'ModerationLabel': {'Confidence': 53.67946243286133,
    'Name': 'Violence',
    'ParentName': ''}},
  {'Timestamp': 1000,
   'ModerationLabel': {'Confidence': 53.67946243286133,
    'Name': 'Weapons',
    'ParentName': 'Violence'}}],
 'ModerationModelVersion': '5.0',
 'ResponseMetadata': {'RequestId': 'c2cbd070-cf81-43ad-b081-1aa4e7fcffb2',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'c2cbd070-cf81-43ad-b081-1aa4e7fcffb2',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '451',
   'date': 'Tue, 15 Nov 2022 18:25:58 GMT'},
  'RetryAttempts': 0}}

# Display List of Unsafe Content in the Video

In [51]:
the_objects = {}

# Potentially unsafe detected in each frame
for obj in get_content_moderation["ModerationLabels"]:
    timestamp = obj["Timestamp"]
    confidence = obj["ModerationLabel"]["Confidence"]
    name = obj["ModerationLabel"]["Name"]
    str_detail = str_detail + "At {} ms: {} (Confidence: {})<br>".format(timestamp, name, round(confidence, 2))
    if name in the_objects:
        cojb = the_objects[name]
        the_objects[name] = {"Name": name, "Count": 1 + cojb["Count"]}
    else:
        the_objects[name] = {"Name": name, "Count": 1}

# Unique objects detected in video
for the_object in the_objects:
    str_overall = str_overall + "Name: {}, Count: {}<br>".format(the_object, the_objects[the_object]["Count"])

# Display results
display(HTML(str_overall))

In [53]:
listui = "<table><tr><td style='vertical-align: top'>{}</td></tr></table>".format(str_detail)
display(HTML(listui))

# Release Resources

In [54]:
%%javascript

try {
    Jupyter.notebook.save_checkpoint();
    Jupyter.notebook.session.delete();
}
catch(err) {
    // NoOp
}

<IPython.core.display.Javascript object>

In [55]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>